In [128]:
import pandas as pd
import numpy as np

In [37]:
df2014 = pd.read_csv("ISP_overall_pop_2014.csv")
df2018 = pd.read_csv("ISP_overall_pop_2018.csv")
df = df2018.merge(df2014,on='HocoNum',how='left')[['HocoNum','population_x','population_y']].reset_index()
df = df.rename(columns={'population_x':'pop2018','population_y':'pop2014'})

In [51]:
new_isp_hoconum = df[pd.isna(df.pop2014)]['HocoNum'].values
new_isp = list(set(new_isp_hoconum))
type2018 = pd.read_csv("ISP_techtype_pop_2018.csv")
type2018[type2018.HocoNum.isin(new_isp)].groupby('TechType').population.sum()

TechType
DOCSIS     1661380
DSL        2032214
FTTH       2518096
Other     36257883
Name: population, dtype: int64

In [118]:
totalpop = df2018.groupby('HocoNum').sum()

In [119]:
FTTH = type2018[type2018.TechType == 'FTTH'].groupby('HocoNum').population.sum().reset_index()
FTTH.rename(columns={'population':'FTTH population'},inplace=True)
quad = totalpop.merge(FTTH,how='left',on='HocoNum').groupby('HocoNum').sum().reset_index()
quad.rename(columns={'population':'total population'},inplace=True)

In [97]:
name = pd.read_csv("HocoNumRef.csv")
name.HocoNum = name.HocoNum.astype('int64',inplace=True)
name = name.drop_duplicates()[['HocoNum','HoldingCompanyName']]
name = name.groupby('HocoNum').first().reset_index()

In [120]:
quadout = quad.merge(name,how='left',on='HocoNum')

In [123]:
quadout[quadout['FTTH population']>quadout['total population']]

,HocoNum,HighSpeed,total population,FTTH population,HoldingCompanyName


In [124]:
quadout.to_csv("ISP_population.csv",index=False)

## Calculate Percentile

In [142]:
np.log10(np.percentile(quadout['total population'],95))

5.718419223214115

In [143]:
np.log10(np.percentile(quadout['FTTH population'],95))

4.736920526863201

In [156]:
type2018_2 = type2018.groupby(['HocoNum','TechType']).sum().reset_index().merge(name,how='left',on='HocoNum')

In [161]:
type2018_2.to_csv('ISP_overall_pop(nospeed)_2018.csv',index=False)

In [158]:
type2014 = pd.read_csv("ISP_techtype_pop_2014.csv")
type2014_2 = type2014.groupby(['HocoNum','TechType']).sum().reset_index().merge(name,how='left',on='HocoNum')
type2014_2.to_csv('ISP_overall_pop(nospeed)_2014.csv',index=False)